In [82]:
#pip install pdfminer.six


from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTRect, LTTextBox, LTTextBoxHorizontal, LTTextLine, LTTextLineHorizontal, LTChar
import os
import configparser

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

class PdfExtractor:
    def __init__(self):
            self.config = configparser.ConfigParser()
            self.config.read('retail.ini')
    
            self.sparkSession = SparkSession \
            .builder \
            .appName("encans") \
            .getOrCreate()
            
            self.sqlContext = SQLContext(self.sparkSession)

            
            '''
            self.logger = logging.getLogger('Retail')
            logfile = self.config.get('dir', 'datadir') + 'indeed_' + str(datetime.now().date()) + '.log'
            hdlr = logging.FileHandler(logfile)
            formatter = logging.Formatter('%(asctime)s -- %(funcName)s -- %(levelname)s -- %(message)s')
            hdlr.setFormatter(formatter)
            self.logger.addHandler(hdlr) 
            self.logger.setLevel(logging.INFO)
            '''           
                
                
    ##
    ## Helping funtions
    ##
    
    def flatten(self, lst):
        """Flattens a list of lists"""
        return [subelem for elem in lst for subelem in elem]


    def extract_characters(self, element):
        """
        Recursively extracts individual characters from 
        text elements. 
        """
        TEXT_ELEMENTS = [LTTextBox, LTTextBoxHorizontal, LTTextLine, LTTextLineHorizontal]

        if isinstance(element, LTChar):
            return [element]

        if any(isinstance(element, i) for i in TEXT_ELEMENTS):
            return self.flatten([self.extract_characters(e) for e in element])

        if isinstance(element, list):
            return self.flatten([self.extract_characters(l) for l in element])

        return []
    
    def extract_layout_by_page(self, pdf_path):
        """
        Extracts LTPage objects from a pdf file.

        slightly modified from
        https://euske.github.io/pdfminer/programming.html
        """
        laparams = LAParams()

        fp = open(pdf_path, 'rb')
        parser = PDFParser(fp)
        document = PDFDocument(parser)

        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed

        rsrcmgr = PDFResourceManager()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        layouts = []
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layouts.append(device.get_result())
            
        # Close object
        fp.close()
        device.close()

        return layouts
    
    
    def array_to_string(array):
        """ Converte an Array to String"""
        return ''.join([str(elem) for elem in array]) 

    def main(self):
        example_file = "catalogue_internet_qc_2018-04-28_v2018-04-13.pdf"
        page_layouts = self.extract_layout_by_page(example_file)
        nb_pages = len(page_layouts)
        #objects_on_page = set(type(o) for o in page_layouts[3])
        current_page = page_layouts[19]

        texts = []

        # seperate text and rectangle elements
        for e in current_page:
            if isinstance(e, LTTextBoxHorizontal):
                texts.append(e)

        # sort them into 
        charactersList = self.extract_characters(texts)
        charactersDF = self.sparkSession.createDataFrame(charactersList)
        maxX0 = charactersDF.agg({"x0": "max"}).collect()[0][0]
        minX0 = charactersDF.agg({"x0": "min"}).collect()[0][0]
        midX0 = (maxX0 + minX0)/2
        
        charactersDF = charactersDF.filter(charactersDF.x0 < midX0) \
                        .union(charactersDF.filter(charactersDF.x0 > midX0))
        
        #charactersDF.printSchema()
        df = charactersDF.groupBy("y0").agg(f.collect_list("_text")).orderBy(charactersDF.y0.desc())
        array_to_string_udf = f.udf(array_to_string,StringType())
        df = df.withColumn('text', array_to_string_udf(df["collect_list(_text)"]))
        df = df.drop('collect_list(_text)')
        #df.write.csv('encans2.csv')
    
        df.printSchema()
        df.show(100)
        
        


In [83]:
pdf = PdfExtractor()
pdf.main()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39615)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39615)

In [84]:
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

def array_to_string(array):
    return ''.join([str(elem) for elem in array]) 



sparkSession = SparkSession \
            .builder \
            .appName("encans") \
            .getOrCreate()
            
sqlContext = SQLContext(sparkSession)
l = [[1, 'a'], [2, 'b'], [2, 'c'], [2, 'd'], [1, 'e'] ]

df = sparkSession.createDataFrame(l)
df2 = df.groupBy("_1").agg(f.collect_list("_2"))
df2 = df2.withColumn('_x', df2['collect_list(_2)'])

#array_to_string_udf = f.udf(array_to_string,StringType())
#df3 = df2.withColumn('column_as_str', array_to_string_udf(df2["collect_list(_2)"]))

df2 = df2.filter(df2._1 < 2)

df2.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39615)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39615)